<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

I followed along during the lecture, so here is my replicated lesson code
https://colab.research.google.com/drive/146mo9BV7uTXuNMgKmeF9k4K85ebK9k-N

In [1]:
##### Begin Working Here #####

import pandas as pd
import numpy as np

# Download the lesson data again
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2020-01-14 20:37:11--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 64.48.1.20
Connecting to resources.lendingclub.com (resources.lendingclub.com)|64.48.1.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip’

LoanStats_2018Q4.cs     [         <=>        ]  21.72M   429KB/s    in 53s     

2020-01-14 20:38:05 (420 KB/s) - ‘LoanStats_2018Q4.csv.zip’ saved [22776791]



In [2]:
!unzip LoanStats_2018Q4.csv.zip

Archive:  LoanStats_2018Q4.csv.zip
  inflating: LoanStats_2018Q4.csv    


In [3]:
# Redoing some of the lecture code in the next several cells
# Read in the CSV
loan_df = pd.read_csv('LoanStats_2018Q4.csv', skiprows = 1, skipfooter = 2)

# Drop unused columns
loan_df = loan_df.drop(['url', 'member_id', 'desc', 'id'], axis=1)

loan_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,6000,6000,6000.0,36 months,14.47%,206.44,C,C2,Manager,10+ years,RENT,55000.0,Verified,Dec-2018,Charged Off,n,credit_card,Credit card refinancing,104xx,NY,10.65,0,Jun-2006,0,NaN,NaN,6,0,4196,91.2%,8,w,0.00,0.00,1106.520000,1106.52,546.14,272.38,0.0,288.0,...,100.0,0,0,15800,12514,4300,11200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Oct-2019,ACTIVE,Oct-2019,3479.0,60.01,24.0
1,24000,24000,24000.0,36 months,10.33%,778.14,B,B1,Maintenance Manager,10+ years,MORTGAGE,100000.0,Not Verified,Dec-2018,Current,n,debt_consolidation,Debt consolidation,606xx,IL,19.14,0,Oct-1996,3,39.0,NaN,14,0,8547,27.7%,28,w,16807.30,16807.30,9317.020000,9317.02,7192.70,2124.32,0.0,0.0,...,16.7,0,0,335121,50686,25900,61514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,30000,30000,30000.0,36 months,15.02%,1040.26,C,C3,Firefighter,10+ years,RENT,90000.0,Verified,Dec-2018,Fully Paid,n,debt_consolidation,Debt consolidation,681xx,NE,22.23,0,May-2007,1,NaN,NaN,8,0,25754,48%,23,w,0.00,0.00,30883.673712,30883.67,30000.00,883.67,0.0,0.0,...,50.0,0,0,89849,51370,41600,36249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,8000,8000,8000.0,36 months,12.98%,269.48,B,B5,Controller,4 years,MORTGAGE,95000.0,Source Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,480xx,MI,9.44,0,Mar-2009,2,73.0,NaN,7,0,11434,33.3%,20,w,1707.34,1707.34,6855.630000,6855.63,6292.66,562.97,0.0,0.0,...,20.0,0,0,271550,25150,34300,14000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,22400,22400,22400.0,60 months,16.14%,546.40,C,C4,Paramedic/ Fire Fighter,10+ years,OWN,56000.0,Source Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,234xx,VA,18.47,0,Dec-2004,0,75.0,NaN,15,0,13555,38.2%,22,w,19230.91,19230.91,6526.670000,6526.67,3169.09,3357.58,0.0,0.0,...,25.0,0,0,253539,32070,28200,23364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#Fix the interest rate column
def convert_percentage_to_float(str_value):
  return float(str_value.strip('%').strip())

In [0]:
loan_df['int_rate_cleaned'] = loan_df['int_rate'].apply(convert_percentage_to_float)

In [0]:
#Fix job titles
def clean_title(job_title):
  if isinstance(job_title, str):
    return job_title.title().strip()
  else:
    return 'Unkown'

In [0]:
loan_df['emp_title'] = loan_df['emp_title'].apply(clean_title)

In [0]:
# change some columns to datetime types
loan_df['issue_d'] = pd.to_datetime(loan_df['issue_d'], infer_datetime_format = True)
loan_df['earliest_cr_line'] = pd.to_datetime(loan_df['earliest_cr_line'], infer_datetime_format = True)

In [10]:
loan_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_cleaned
0,6000,6000,6000.0,36 months,14.47%,206.44,C,C2,Manager,10+ years,RENT,55000.0,Verified,2018-12-01,Charged Off,n,credit_card,Credit card refinancing,104xx,NY,10.65,0,2006-06-01,0,NaN,NaN,6,0,4196,91.2%,8,w,0.00,0.00,1106.520000,1106.52,546.14,272.38,0.0,288.0,...,0,0,15800,12514,4300,11200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Oct-2019,ACTIVE,Oct-2019,3479.0,60.01,24.0,14.47
1,24000,24000,24000.0,36 months,10.33%,778.14,B,B1,Maintenance Manager,10+ years,MORTGAGE,100000.0,Not Verified,2018-12-01,Current,n,debt_consolidation,Debt consolidation,606xx,IL,19.14,0,1996-10-01,3,39.0,NaN,14,0,8547,27.7%,28,w,16807.30,16807.30,9317.020000,9317.02,7192.70,2124.32,0.0,0.0,...,0,0,335121,50686,25900,61514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,10.33
2,30000,30000,30000.0,36 months,15.02%,1040.26,C,C3,Firefighter,10+ years,RENT,90000.0,Verified,2018-12-01,Fully Paid,n,debt_consolidation,Debt consolidation,681xx,NE,22.23,0,2007-05-01,1,NaN,NaN,8,0,25754,48%,23,w,0.00,0.00,30883.673712,30883.67,30000.00,883.67,0.0,0.0,...,0,0,89849,51370,41600,36249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,15.02
3,8000,8000,8000.0,36 months,12.98%,269.48,B,B5,Controller,4 years,MORTGAGE,95000.0,Source Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,480xx,MI,9.44,0,2009-03-01,2,73.0,NaN,7,0,11434,33.3%,20,w,1707.34,1707.34,6855.630000,6855.63,6292.66,562.97,0.0,0.0,...,0,0,271550,25150,34300,14000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98
4,22400,22400,22400.0,60 months,16.14%,546.40,C,C4,Paramedic/ Fire Fighter,10+ years,OWN,56000.0,Source Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,234xx,VA,18.47,0,2004-12-01,0,75.0,NaN,15,0,13555,38.2%,22,w,19230.91,19230.91,6526.670000,6526.67,3169.09,3357.58,0.0,0.0,...,0,0,253539,32070,28200,23364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,16.14


# Objective: Convert the term column from string to integer.

In [21]:
loan_df['term'].value_counts()

 36 months    88171
 60 months    40226
Name: term, dtype: int64

In [14]:
loan_df['term'][0]

' 36 months'

In [0]:
# Create the function
def clean_term_column(term):
  return int(term[0:4].strip()) # slice to first 4 characters and strip the spaces before and after

    # Gets first 4 so that data with 3 digit months still works, even if this set doesn't have any.  
    # Example: ' 120 months' -> ' 120' -> 120

In [22]:
# Test the function
clean_term_column(loan_df['term'][0])

36

In [0]:
# Apply to the column
loan_df['term'] = loan_df['term'].apply(clean_term_column)

In [28]:
loan_df['term'].value_counts()

36    88171
60    40226
Name: term, dtype: int64

In [29]:
type(loan_df['term'][0])

numpy.int64

# Objective: Make a column named loan_status_is_great

It should contain the integer 1 if loan_status is "Current" or "Fully Paid." Else it should contain the integer 0.

In [31]:
# Check data
loan_df['loan_status'].value_counts()

Current               97694
Fully Paid            21476
Charged Off            5260
Late (31-120 days)     2656
In Grace Period         784
Late (16-30 days)       524
Default                   3
Name: loan_status, dtype: int64

In [35]:
loan_df['loan_status'].head()

0    Charged Off
1        Current
2     Fully Paid
3        Current
4        Current
Name: loan_status, dtype: object

In [36]:
loan_df['loan_status'][1] # leading spaces? nope

'Current'

In [0]:
# Define the function
def check_loan_status(status):
  if status == 'Fully Paid' or status == 'Current':
    return 1
  else:
    return 0

In [46]:
# Test the function
examples = ['Current', 'Fully Paid', 'Charged Off', 'something else', '']

for ex in examples:
  print(check_loan_status(ex))

1
1
0
0
0


In [0]:
# Create the new column
loan_df['loan_status_is_great'] = loan_df['loan_status'].apply(check_loan_status)

In [48]:
loan_df[['loan_status_is_great', 'loan_status']].head(10)

,loan_status_is_great,loan_status
0,0,Charged Off
1,1,Current
2,1,Fully Paid
3,1,Current
4,1,Current
5,1,Current
6,1,Current
7,1,Current
8,1,Current
9,1,Current


In [49]:
# Expected: 119170 1's (from the earlier count on 'loan_status')
loan_df['loan_status_is_great'].value_counts()

1    119170
0      9227
Name: loan_status_is_great, dtype: int64

# Objective: Make last_pymnt_d_month and last_pymnt_d_year columns

In [58]:
loan_df['last_pymnt_d'].head()

0    Apr-2019
1    Dec-2019
2    Feb-2019
3    Dec-2019
4    Dec-2019
Name: last_pymnt_d, dtype: object

In [69]:
loan_df['last_pymnt_d'].isnull().sum()

159

In [0]:
# Convert that to datetime
loan_df['last_pymnt_d'] = pd.to_datetime(loan_df['last_pymnt_d'], infer_datetime_format = True)

In [0]:
# Add the new columns
loan_df['last_pymnt_d_month'] = loan_df['last_pymnt_d'].dt.month
loan_df['last_pymnt_d_year'] = loan_df['last_pymnt_d'].dt.year

In [63]:
# View the columns, new ones end up as floats because last_pymnt_d had NaN values
loan_df[['last_pymnt_d', 'last_pymnt_d_month', 'last_pymnt_d_year']].head()

,last_pymnt_d,last_pymnt_d_month,last_pymnt_d_year
0,2019-04-01,4.0,2019.0
1,2019-12-01,12.0,2019.0
2,2019-02-01,2.0,2019.0
3,2019-12-01,12.0,2019.0
4,2019-12-01,12.0,2019.0


# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

# Goal:
There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.

In [70]:
loan_df['revol_util'].head()

0    91.2%
1    27.7%
2      48%
3    33.3%
4    38.2%
Name: revol_util, dtype: object

In [71]:
loan_df['revol_util'].isnull().sum()

156

In [100]:
# let's find a NaN so we can test with it
loan_df[loan_df['revol_util'].isnull()].head(3)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_cleaned,loan_status_is_great,last_pymnt_d_month,last_pymnt_d_year
240,2000,2000,2000.0,36,17.97%,72.28,D,D1,Account Manager,4 years,RENT,51000.0,Source Verified,2018-12-01,Current,n,debt_consolidation,Debt consolidation,606xx,IL,2.40,0,2004-11-01,1,NaN,NaN,1,0,0,NaN,9,w,1448.04,1448.04,864.36,864.36,551.96,312.40,0.0,0.0,...,854,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,17.97,1,12.0,2019.0
5141,5000,5000,5000.0,36,10.72%,163.04,B,B2,President,6 years,MORTGAGE,30000.0,Verified,2018-12-01,Current,n,home_improvement,Home improvement,280xx,NC,5.08,0,1994-07-01,0,56.0,NaN,2,0,0,NaN,11,w,3507.61,3507.61,1950.52,1950.52,1492.39,458.13,0.0,0.0,...,3331,0,4200,17705.0,Jul-1994,0.0,3.0,2.0,NaN,1.0,9.0,2.0,0.0,1.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,10.72,1,12.0,2019.0
5272,3500,3500,3500.0,36,13.56%,118.88,C,C1,Nursing Supervisor,10+ years,RENT,32000.0,Source Verified,2018-12-01,Current,n,medical,Medical expenses,911xx,CA,39.65,0,2004-05-01,1,28.0,NaN,2,0,0,NaN,28,w,2487.69,2487.69,1421.29,1421.29,1012.31,408.98,0.0,0.0,...,4457,0,20045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,13.56,1,12.0,2019.0


In [94]:
loan_df['revol_util'][240]

nan

In [0]:

# define a function for it
def clean_percentages(cell):
  if pd.isna(cell):
    return np.nan
  else:
    return float(cell.strip().strip('%'))

In [96]:
# test the function
clean_percentages(loan_df['revol_util'][0])

91.2

In [97]:
clean_percentages(loan_df['revol_util'][240])

nan

In [0]:
# apply to the column
loan_df['revol_util'] = loan_df['revol_util'].apply(clean_percentages)

In [99]:
loan_df['revol_util'].head()

0    91.2
1    27.7
2    48.0
3    33.3
4    38.2
Name: revol_util, dtype: float64

# Goal:
Modify the emp_title column to replace titles with 'Other' if the title is not in the top 20.

In [103]:
# Save this list to check against
top_jobs = loan_df['emp_title'].value_counts()[:20]

top_jobs

Unkown                20942
Teacher                2557
Manager                2395
Registered Nurse       1417
Driver                 1258
Supervisor             1160
Truck Driver            920
Rn                      834
Office Manager          804
Sales                   803
General Manager         791
Project Manager         720
Owner                   625
Director                523
Operations Manager      518
Sales Manager           500
Police Officer          440
Nurse                   424
Technician              420
Engineer                412
Name: emp_title, dtype: int64

In [0]:
# Create a function
def filter_job_titles(title):
  if title in top_jobs:
    return title
  else:
    return 'Other'

In [108]:
# Test it
example = ['Driver', 'Operations Manager', 'Pilot', '', 0]

for ex in example:
  print(filter_job_titles(ex))

Driver
Operations Manager
Other
Other
Other


In [0]:
# Apply it
loan_df['emp_title'] = loan_df['emp_title'].apply(filter_job_titles)

In [110]:
loan_df['emp_title'].value_counts()

Other                 89934
Unkown                20942
Teacher                2557
Manager                2395
Registered Nurse       1417
Driver                 1258
Supervisor             1160
Truck Driver            920
Rn                      834
Office Manager          804
Sales                   803
General Manager         791
Project Manager         720
Owner                   625
Director                523
Operations Manager      518
Sales Manager           500
Police Officer          440
Nurse                   424
Technician              420
Engineer                412
Name: emp_title, dtype: int64

In [112]:
# Should now be 21 different titles, the original top 20 and 'Other'
len(loan_df['emp_title'].value_counts())

21

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01